---
# 올리브영 제품군 크롤링
* 크림: https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010015&fltDispCatNo=&prdSort=01&pageIdx=3
* 로션: https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010016&fltDispCatNo=&prdSort=01&pageIdx=3
* 스킨: https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010013&fltDispCatNo=&prdSort=01&pageIdx=3
* 에센스앰플: https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100140001&fltDispCatNo=&prdSort=01&pageIdx={page}
---

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# 전체 페이지

In [2]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service,options=options)

time.sleep(0.2)

prd_items = []
prd_brands = []
prd_names = []
prd_prices = []
prd_infos = []
prd_thumbs = [] 
try: 
    for page in range(1, 3):
        url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010015&fltDispCatNo=&prdSort=01&pageIdx={page}"
        driver.get(url)

        time.sleep(1.0)
        prd_list = driver.find_element(By.ID,"Contents")
        # prd = prd_list.find_element(By.CLASS_NAME,"cate_prd_list")      # 한 페이지에 이 클래스 이름을 가진 ul이 여러 개고 그 ul마다 각각 상품 네개씩 있음
        prd = prd_list.find_elements(By.CLASS_NAME,"prd_info")              # 그냥 바로 상품 각각 찾기

        for i in range(len(prd)):
            prds = driver.find_elements(By.CLASS_NAME, "prd_info")  # Refresh the list of prd elements
            each_prd = prds[i]
            each_prd.click()

            info_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "buyInfo")))
            info_btn.click()

            prd_items = driver.find_element(By.CLASS_NAME, "right_area")
            prd_brands.append(prd_items.find_element(By.ID, "moveBrandShop").text)    
            prd_name = prd_items.find_element(By.CLASS_NAME, "prd_name").text
            prd_names.append(prd_name)
            prd_prices.append(prd_items.find_element(By.CLASS_NAME, "price").text.split('\n')[0])
            
            
            prd_images_area = driver.find_element(By.ID, "prd_thumb_list")
            thumbs = prd_images_area.find_elements(By.TAG_NAME, "li")
            imageNames = []
            for idx, li in enumerate(thumbs):
                
                imgs = li.find_element(By.TAG_NAME, "img")
                imgsrc = imgs.get_attribute("src")
                imgres = requests.get(imgsrc)

                imageName = f"./prdimgs/page_{page}_prd_{i}_thumb{idx + 1}.jpg"
                with open(imageName, "wb") as f:
                    f.write(imgres.content)
                imageNames.append(imageName)
                
            prd_thumbs.append(imageNames)
            prd_info = WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.ID, "artcInfo")))
            prd_infos.append(prd_info.text.split("\n"))
            driver.back()
            time.sleep(0.2)
finally:
    driver.quit()

In [4]:
# page=2까지
import pandas as pd
cream = pd.DataFrame({"제품명":prd_names, "브랜드":prd_brands, "가격":prd_prices,'상품정보':prd_infos,'썸네일':prd_thumbs})
cream

,제품명,브랜드,가격,상품정보,썸네일
0,[NEW/리뷰이벤트] 에스트라 아토베리어365 크림 80ml,에스트라,"33,000","[내용물의 용량 또는 중량 80ml, 제품 주요 사양 모든 피부용, 사용기한(또는 ...",[./prdimgs/page_1_prd_0_thumb1.jpg]
1,라로슈포제 시카플라스트 밤B5+ 200ml 기획 (+시카토너 50ml+시카밤 3ml...,라로슈포제,"68,000","[내용물의 용량 또는 중량 200ml + 50ml + 3ml * 2, 제품 주요 사...",[./prdimgs/page_1_prd_1_thumb1.jpg]
2,[2세대 천만크림] 닥터지 레드 블레미쉬 클리어 수딩 크림 70ml+30ml 세트,닥터지,"38,000","[내용물의 용량 또는 중량 크림 70ml(jar) + 30ml(tube), 제품 주...","[./prdimgs/page_1_prd_2_thumb1.jpg, ./prdimgs/..."
3,[더블기획] 피지오겔 DMT 페이셜 크림 100ml+100ml 증정 기획,피지오겔,"59,000","[내용물의 용량 또는 중량 100ml+100ml, 제품 주요 사양 모든피부용, 사용...","[./prdimgs/page_1_prd_3_thumb1.jpg, ./prdimgs/..."
4,[2023어워즈/1+1] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 어워즈 ...,에스네이처,"43,000","[내용물의 용량 또는 중량 60ml*2개입, 제품 주요 사양 모든 피부 타입, 사용...","[./prdimgs/page_1_prd_4_thumb1.jpg, ./prdimgs/..."
5,[나도PICK/100ml+100ml] 닥터지 더모이스처 배리어 D 인텐스 크림 10...,닥터지,"45,000","[내용물의 용량 또는 중량 크림 100ml+100ml, 제품 주요 사양 모든 피부용...","[./prdimgs/page_1_prd_5_thumb1.jpg, ./prdimgs/..."
6,[1월 올영픽] 아비브 부활초 크림 뉴트리션 튜브 75ml 기획 (+30ml 추가 증정),아비브,"35,000","[내용물의 용량 또는 중량 75ml(+30ml 증정), 제품 주요 사양 모든 피부용...","[./prdimgs/page_1_prd_6_thumb1.jpg, ./prdimgs/..."
7,[단독대용량] 라운드랩 자작나무 수분 크림 120ml 기획 (+수분패드 4매),라운드랩,"42,000",[내용물의 용량 또는 중량 [본품] 라운드랩 자작나무 수분 크림 120ml [증정품...,"[./prdimgs/page_1_prd_7_thumb1.jpg, ./prdimgs/..."
8,[단독기획] 웰라쥬 리얼 히알루로닉 100 크림 50ml 1+1 기획,웰라쥬,"39,000","[내용물의 용량 또는 중량 50ml+50ml, 제품 주요 사양 모든피부타입, 사용기...","[./prdimgs/page_1_prd_8_thumb1.jpg, ./prdimgs/..."
9,[단독기획] 토리든 다이브인 저분자 히알루론산 수딩 크림 100ml 기획(+20ml),토리든,"24,000",[내용물의 용량 또는 중량 [본품] 다이브인 수딩크림 100ml [증정] 다이브인 ...,"[./prdimgs/page_1_prd_9_thumb1.jpg, ./prdimgs/..."


In [5]:
category = '크림'
cream.to_csv(f'./data/02.데이터수집_01크롤링_올리브영_제품_{category}.csv', index=False)

In [6]:
cream.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     48 non-null     object
 1   브랜드     48 non-null     object
 2   가격      48 non-null     object
 3   상품정보    48 non-null     object
 4   썸네일     48 non-null     object
dtypes: object(5)
memory usage: 2.0+ KB
